In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Trees_Example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 19:20:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = spark.read.csv("College.csv", inferSchema=True, header=True)

In [3]:
data.show(truncate=False)

+---------------------------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|School                                 |Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+---------------------------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian University           |Yes    |1660|1232  |721   |23       |52       |2885       |537        |7440    |3300      |450  |2200    |70 |78      |18.1     |12         |7041  |60       |
|Adelphi University                     |Yes    |2186|1924  |512   |16       |29       |2683       |1227       |12280   |6450      |750  |1500    |29 |30      |12.2     |16         |10527 |56       |


In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(
    inputCols=[
        "Apps",
        "Accept",
        "Enroll",
        "Top10perc",
        "Top25perc",
        "F_Undergrad",
        "P_Undergrad",
        "Outstate",
        "Room_Board",
        "Books",
        "Personal",
        "PhD",
        "Terminal",
        "S_F_Ratio",
        "perc_alumni",
        "Expend",
        "Grad_Rate",
    ],
    outputCol="features",
)

In [8]:
output = assembler.transform(data)

In [9]:
from pyspark.ml.feature import StringIndexer

In [10]:
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")

In [11]:
output_fixed = indexer.fit(output).transform(output)

In [12]:
output_fixed.show()

24/03/14 19:27:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|            features|PrivateIndex|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+------------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|[1660.0,1232.0,72...|         0.0|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30

In [13]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [16]:
final_data = output_fixed.select("features", "PrivateIndex")

In [17]:
train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=20)

In [18]:
from pyspark.ml.classification import (
    DecisionTreeClassifier,
    GBTClassifier,
    RandomForestClassifier,
)

In [19]:
from pyspark.ml import Pipeline

In [20]:
dtc = DecisionTreeClassifier(labelCol="PrivateIndex", featuresCol="features")
rfc = RandomForestClassifier(labelCol="PrivateIndex", featuresCol="features")
gbt = GBTClassifier(labelCol="PrivateIndex", featuresCol="features")

In [21]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [22]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
my_binary_eval = BinaryClassificationEvaluator(labelCol="PrivateIndex")

In [25]:
print("DTC")
print(my_binary_eval.evaluate(dtc_preds))

DTC
0.9399020694752402


In [26]:
print("RFC")
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9763488543976342


In [27]:
print("GBT")
print(my_binary_eval.evaluate(gbt_preds))

GBT
0.9376847745750183


24/03/14 19:53:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/03/14 19:53:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [28]:
my_binary_eval2 = BinaryClassificationEvaluator(
    labelCol="PrivateIndex", rawPredictionCol="prediction"
)

In [29]:
print("GBT")
print(my_binary_eval2.evaluate(gbt_preds))

GBT
0.8921840354767184


In [34]:
rfc2 = RandomForestClassifier(
    labelCol="PrivateIndex", featuresCol="features", numTrees=150
)
rfc_model2 = rfc2.fit(train_data)
rfc_preds2 = rfc_model2.transform(test_data)

In [35]:
print("RFC")
print(my_binary_eval.evaluate(rfc_preds2))

RFC
0.9742239467849211


In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [37]:
acc_eval = MulticlassClassificationEvaluator(
    labelCol="PrivateIndex", metricName="accuracy"
)

In [38]:
rfc_acc = acc_eval.evaluate(rfc_preds)
rfc_acc

0.9304347826086956

In [39]:
rfc_acc2 = acc_eval.evaluate(rfc_preds2)
rfc_acc2

0.9347826086956522